# 7. use tools 使用工具

首先将zerollama目录加入python path

In [1]:
import os
import sys
from pathlib import Path
pwd = Path(os.getcwd())
sys.path.append(str(pwd.parent.parent.parent))
os.chdir(str(pwd.parent.parent.parent))

导入模块

In [2]:
import json
from zerollama.agents.use_tool.agent_use_tools import AgentUseTools

定义第一个工具 查询当前天气，当然为了测试，温度是写死的。

In [3]:
def get_current_temperature(location: str, unit: str="celsius") -> str:
    """
    Get the current temperature at a location.

    Args:
        location: The location to get the temperature for, in the format "City, Country"
        unit: The unit to return the temperature in. (choices: ["celsius", "fahrenheit"])
    Returns:
        The current temperature at the specified location in the specified units.
    """
    if "chicago" in location.lower():
        return json.dumps({"location": "Chicago", "temperature": "13", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "55", "unit": unit})
    elif "new york" in location.lower():
        return json.dumps({"location": "New York", "temperature": "11", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

第二个工具 查询当前风速，当然风速也是写死的。

In [4]:
def get_current_wind_speed(location: str) -> float:
    """
    Get the current wind speed in km/h at a given location.

    Args:
        location: The location to get the temperature for, in the format "City, Country"
    Returns:
        The current wind speed at the given location in km/h, as a float.
    """
    return 6.  # A real function should probably actually get the wind speed!

我们是用NousResearch/Hermes-2-Pro-Llama-3-8B，据说是同尺寸模型里，工具调用最好的之一。

In [5]:
model = "NousResearch/Hermes-2-Pro-Llama-3-8B"

可以使用openai和zerollama接口形式

In [6]:
for llm_config in [
    {"type": "zerollama", "model": model, "global_priority": False},
    {"type": "openai", "model": model, "base_url": 'http://localhost:8080/v1/', "global_priority": False},
]:
    agent = AgentUseTools(
        system_message="You are a bot that responds to weather queries. You should reply with the unit used in the queried location.",
        tools=[get_current_temperature, get_current_wind_speed],
        llm_config=llm_config
    )
    print(llm_config["type"])

    messages = [
        {"role": "user", "content": "What is the current temperature of New York, San Francisco and Chicago?"}
    ]

    reply = agent.generate_reply(messages)

    print(reply)

zerollama
The current temperatures are:
- New York: 11°C
- San Francisco: 55°C
- Chicago: 13°C
openai
The current temperature in New York is 11°F, in San Francisco it is 55°F, and in Chicago it is 13°F.


# 总结
能使用工具为llm如虎添翼。